In [ ]:
!pip install --quiet bitsandbytes
!pip install --quiet git+https://github.com/huggingface/transformers.git # Install latest version of transformers
!pip install --quiet accelerate
!pip install --quiet sentencepiece

     |████████████████████████████████| 62.5 MB 1.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.6 MB 5.1 MB/s 
     |████████████████████████████████| 163 kB 77.7 MB/s 
     |████████████████████████████████| 148 kB 5.0 MB/s 
     |████████████████████████████████| 1.3 MB 5.0 MB/s 


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Oct 11 15:43:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    44W / 350W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Local loading

In [ ]:
!git lfs install
!git clone -b sharded --single-branch https://huggingface.co/facebook/incoder-6B

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'incoder-6B'...
remote: Enumerating objects: 45, done.
remote: Total 45 (delta 0), reused 0 (delta 0), pack-reused 45
Unpacking objects: 100% (45/45), done.
tcmalloc: large alloc 1471086592 bytes == 0x55ca762be000 @  0x7f3d6a0842a4 0x55ca3919511f 0x55ca3917225b 0x55ca39126f33 0x55ca390cb22a 0x55ca390cb6e6 0x55ca390e8451 0x55ca390e89e9 0x55ca390e8f13 0x55ca3918de12 0x55ca3902f162 0x55ca39015a65 0x55ca39016725 0x55ca3901572a 0x7f3d693cbc87 0x55ca3901577a
tcmalloc: large alloc 2206621696 bytes == 0x55cacddae000 @  0x7f3d6a0842a4 0x55ca3919511f 0x55ca3917225b 0x55ca39126f33 0x55ca390cb22a 0x55ca390cb6e6 0x55ca390e8451 0x55ca390e89e9 0x55ca390e8f13 0x55ca3918de12 0x55ca3902f162 0x55ca39015a65 0x55ca39016725 0x55ca3901572a 0x7f3d693cbc87 0x55ca3901577a
tcmalloc: large alloc 3309936640 bytes == 0x55cb51614000 @  0x7f3d6a0842a

##  InCoder 1B

In [ ]:
!git lfs install
!git clone https://huggingface.co/facebook/incoder-1B

then we can initialize the model with

In [ ]:
from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM

checkpoint = "facebook/incoder-1B"
config = AutoConfig.from_pretrained(checkpoint)

with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

and load the checkpoint we just downloaded with:

In [ ]:
from accelerate import load_checkpoint_and_dispatch

model = load_checkpoint_and_dispatch(
    model, "incoder-1B", device_map="auto"
)

In [ ]:
model.hf_device_map

In [ ]:
import torch
torch.cuda.empty_cache()

print(f'available devices: {torch.cuda.device_count()}')
print(f'current device: { torch.cuda.current_device()}')

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
inputs = tokenizer("def", return_tensors="pt")
inputs = inputs.to(0)
output = model.generate(inputs["input_ids"])
tokenizer.decode(output[0].tolist())

# HF Remote Repository Load



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [5]:
# We need sharded weights otherwise we get CPU OOM errors
model_name = "incoder-6B"
model_id=f"facebook/{model_name}"

model = AutoModelForCausalLM.from_pretrained("facebook/incoder-6B", revision="sharded", low_cpu_mem_usage=True)
#model = AutoModelForCausalLM.from_pretrained(model_id, low_cpu_mem_usage=True)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Downloading:   0%|          | 0.00/611 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/40.9k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.93G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.54G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/212 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

In [6]:
model.get_memory_footprint() / 10**9

26.637942784

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model.cuda()

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(50518, 4096, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=True)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=4096, out_features=16384, bias=True)
        (fc2): Linear(in_features=16384, out_features=4096, bias=True)
        (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
      )
      (1): XGLMDecoderLayer(
        (self_attn): XGLMAttention(
          (k_p

In [8]:
from typing import List

import torch
import tokenizers
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
tokenizers_version = tuple(int(n) for n in tokenizers.__version__.split('.'))
if tokenizers_version < (0, 12, 1):
    print("warning: Your tokenizers version looks old and you will likely have formatting issues. We recommend installing tokenizers >= 0.12.1")

# set BIG_MODEL to use the 6.7B parameter model
BIG_MODEL = False

# use a GPU
CUDA = True

# print intermediate outputs of infilling
VERBOSE = False
"""
if BIG_MODEL:
    model_name = "facebook/incoder-6B"

    # the arguments added below will load a half precision version of the model,
    # which requires less RAM than loading the full float32 version.  this 
    # should fit in ~16GB of RAM
    # NOTE: half precision should *not* be used if you plan to fine-tune the
    # model. You'll need full precision and a lot of GPU memory. We have not
    # tested fine-tuning in `transformers` (the model was trained in fairseq)
    if CUDA:
        kwargs = dict(
            revision="float16", 
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
        )
    else:
        kwargs = dict(
            low_cpu_mem_usage=True,
        )
else:
    model_name = "facebook/incoder-1B"
    kwargs = {}

print("loading model")
model = AutoModelForCausalLM.from_pretrained(model_name, **kwargs)
print("loading tokenizer")
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("loading complete")
"""
if CUDA:
    # if you plan to fine-tune the model, you should not use half precision.
    model = model.half().cuda()

# signals the start of a document
BOS = "<|endoftext|>"
# signals the end of a generated infill
EOM = "<|endofmask|>"

def make_sentinel(i):
    # signals (1) a location to insert an infill and (2) the start of the infill generation
    return f"<|mask:{i}|>"

def generate(input: str, max_to_generate: int=128, temperature: float=0.2):
    """
    Do standard left-to-right completion of the prefix `input` by sampling from the model
    """
    input_ids = tokenizer(input, return_tensors="pt").input_ids
    if CUDA:
        input_ids = input_ids.cuda()
    max_length = max_to_generate + input_ids.flatten().size(0)
    if max_length > 2048:
        print("warning: max_length {} is greater than the context window {}".format(max_length, 2048))
    with torch.no_grad():
        output = model.generate(input_ids=input_ids, do_sample=True, top_p=0.95, temperature=temperature, max_length=max_length)
    # pass clean_up_tokenization_spaces=False to avoid removing spaces before punctuation, e.g. "from ." -> "from."
    detok_hypo_str = tokenizer.decode(output.flatten(), clean_up_tokenization_spaces=False)
    if detok_hypo_str.startswith(BOS):
        detok_hypo_str = detok_hypo_str[len(BOS):]
    return detok_hypo_str

def infill(parts: List[str], max_to_generate: int=128, temperature: float=0.2, extra_sentinel: bool=True, max_retries: int=1):
    """
    Generate infills to complete a partial document, e.g.
    [A C E] -> [A B C D E], where B and D are infills that have been generated.
    parts: List[str]. list of parts of the document. One string will be
            inserted in between each element, i.e. infilling N-1 locations for a list
            of length N.
    max_to_generate: int. maximum number of tokens to generate. Keep in mind
            that the model context size is 2048.
    temperature: float. temperature parameter for sampling.
    extra_sentinel: bool. we recommend setting this to True, as it makes it
            easier for the model to end generated infills. See the footnote in 
            section 2.2 of our paper for details.
    max_retries: int. if > 1, use rejection sampling to keep sampling infills until
            all infills sample a completion token.
    returns a dictionary containing the following:
        text:  str, the completed document (with infills inserted)
        parts:  List[str], length N. Same as passed to the method
        infills:  List[str], length N-1. The list of infills generated
        retries_attempted:  number of retries used (if max_retries > 1)
    """
    assert isinstance(parts, list)
    retries_attempted = 0
    done = False

    while (not done) and (retries_attempted < max_retries):
        retries_attempted += 1

        if VERBOSE:
            print(f"retry {retries_attempted}")
        
        ## (1) build the prompt
        if len(parts) == 1:
            prompt = parts[0]
        else:
            prompt = ""
            # encode parts separated by sentinel
            for sentinel_ix, part in enumerate(parts):
                prompt += part
                if extra_sentinel or (sentinel_ix < len(parts) - 1):
                    prompt += make_sentinel(sentinel_ix)
        
        infills = []
        complete = []

        done = True

        ## (2) generate infills
        for sentinel_ix, part in enumerate(parts[:-1]):
            complete.append(part)
            prompt += make_sentinel(sentinel_ix)
            # TODO: this is inefficient as it requires re-encoding prefixes repeatedly
            completion = generate(prompt, max_to_generate, temperature)
            completion = completion[len(prompt):]
            if EOM not in completion:
                if VERBOSE:
                    print(f"warning: {EOM} not found")
                completion += EOM
                done = False
            completion = completion[:completion.index(EOM) + len(EOM)]
            infilled = completion[:-len(EOM)]
            infills.append(infilled)
            complete.append(infilled)
            prompt += completion
        complete.append(parts[-1])
        text = ''.join(complete)

    if VERBOSE:
        print("generated text:")
        print(prompt)
        print()
        print("parts:")
        print(parts)
        print()
        print("infills:")
        print(infills)
        print()
        print("restitched text:")
        print(text)
        print()
    
    return {
        'text': text, # str, the completed document (with infills inserted)
        'parts': parts, # List[str], length N. Same as passed to the method
        'infills': infills, # List[str], length N-1. The list of infills generated
        'retries_attempted': retries_attempted, # number of retries used (if max_retries > 1)
    } 

def code_to_docstring(max_to_generate=128, temperature=0.2):
    # this will sometimes generate extra functions! this can be avoided by truncating generation when e.g. a """ is produced
    example = '''\
def count_words(filename):
    """ <insert> """
    counts = Counter()
    with open(filename) as file:
        for line in file:
            words = line.split(' ')
            counts.update(words)
    return counts'''

    parts = example.split("<insert>")
    result = infill(parts, max_to_generate=max_to_generate, temperature=temperature)
    print("completed document:")
    print(result["text"])
    return result["text"]

def docstring_to_code(max_to_generate=128, temperature=0.2):
    example = '''\
def <insert>
    """ Count the number of occurrences of each word in the file. """
    <insert>
<|/ file |>'''
    parts = example.split("<insert>")
    result = infill(parts, max_to_generate=max_to_generate, temperature=temperature)
    print("completed document:")
    print(result["text"])
    return result["text"]

In [ ]:
!pip install --quiet datasets

     |████████████████████████████████| 365 kB 37.3 MB/s 
     |████████████████████████████████| 212 kB 91.6 MB/s 
     |████████████████████████████████| 115 kB 71.2 MB/s 
     |████████████████████████████████| 127 kB 73.0 MB/s 


In [ ]:
from datasets import load_dataset

dataset = load_dataset("openai_humaneval")

"""
    <insert>
<|/ file |>'''
"""
# Lambda expression
updated_dataset = dataset.map(lambda x: {"prompt": x["prompt"] + "  <insert>" + "\n" + "<|/ file |>"})

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/164 [00:00<?, ?ex/s]

In [ ]:
print(dataset["test"][0]["prompt"])
print(updated_dataset["test"][0]["prompt"])

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
  <insert>
<|/ file |>


In [ ]:
parts = updated_dataset["test"][0]["prompt"].split("<insert>")

program = infill(parts)

In [ ]:
# Example code generated solution to a given prompt
print(program["text"])

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """
  
    diffs = [abs(a - b) for a, b in zip(numbers[:-1], numbers[1:])]
    diffs.sort()
    return diffs[0] < threshold
<|/ file |>


In [ ]:
print("code to docstring test:")
code_to_docstring()
print()
print("docstring to code test:")
docstring_to_code()

# Dummy pass@k Test

In [ ]:
"""
print("code to docstring test:")

pass_at_100 = [code_to_docstring() for i in range(100)]
code_to_docstring()
print()
print("docstring to code test:")
docstring_to_code()
"""

In [ ]:
print(pass_at_100[0])

In [ ]:
from typing import Iterable, Dict
import gzip
import json
import os

def write_jsonl(filename: str, data: Iterable[Dict], append: bool = False):
    """
    Writes an iterable of dictionaries to jsonl
    """
    if append:
        mode = 'ab'
    else:
        mode = 'wb'
    filename = os.path.expanduser(filename)
    if filename.endswith(".gz"):
        with open(filename, mode) as fp:
            with gzip.GzipFile(fileobj=fp, mode='wb') as gzfp:
                for x in data:
                    gzfp.write((json.dumps(x) + "\n").encode('utf-8'))
    else:
        with open(filename, mode) as fp:
            for x in data:
                fp.write((json.dumps(x) + "\n").encode('utf-8'))

In [ ]:
write_jsonl("./results_pass_at_100.jsonl", pass_at_100)

In [50]:
prompt: str = "from typing import List\n\ndef f1(numbers: List[float], threshold: float) -> bool:\n\"\"\"\nf1: List[float] -> float -> bool\n    assert f1([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert f1([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert f1([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True\n\"\"\"\n    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n\n// Rust f1 translation\nfn has_close_elements(numbers: Vec<f64>, threshold: f64) -> bool {\n    for idx in 0..numbers.len() {\n        for idx2 in 0..numbers.len() {\n            if idx != idx2 {\n                let distance = (numbers[idx] - numbers[idx2]).abs();\n                if distance < threshold {\n                    return true;\n                }\n            }\n        }\n    }\n\n    return false;\n}\n\nfrom typing import List\n\ndef f2(paren_string: str) -> List[str]:\n\"\"\"\nf2: str -> List[str]\n    assert f2('(()()) ((())) () ((())()())') == [\n        '(()())', '((()))', '()', '((())()())'\n    ]\n    assert f2('() (()) ((())) (((())))') == [\n        '()', '(())', '((()))', '(((())))'\n    ]\n    assert f2('(()(())((())))') == [\n        '(()(())((())))'\n    ]\n    assert f2('( ) (( )) (( )( ))') == ['()', '(())', '(()())']\n\"\"\"\n    result = []\n    current_string = []\n    current_depth = 0\n\n    for c in paren_string:\n        if c == '(':\n            current_depth += 1\n            current_string.append(c)\n        elif c == ')':\n            current_depth -= 1\n            current_string.append(c)\n\n            if current_depth == 0:\n                result.append(''.join(current_string))\n                current_string.clear()\n\n    return result\n\n\ndef f3(number: float) -> float:\n    \"\"\" \nf3: float -> float\n    assert f3(3.5) == 0.5\n    assert abs(f3(1.33) - 0.33) < 1e-6\n    assert abs(f3(123.456) - 0.456) < 1e-6\n    \"\"\"\n    return number - int(number)\n\ndef f4(a, b):\n    \"\"\"\nComplete the function that takes two integers and returns the product of their unit digits.\nf4: int -> int -> int\n    assert f4(148, 412) == 16,             \n    assert f4(19, 28) == 72, \n    assert f4(2020, 1851) == 0\n    assert f4(3,3) == 9\n    assert f4(12,33) == 6\n    \"\"\"\n    return (a%10)*(b%10)\n\ndef f5(xs: List[int]) -> List[int]:\n    \"\"\" \nxs represent coefficients of a polynomial.\nf5: List[int] -> List[int]\n    assert f5([3, 1, 2, 4, 5]) == [1, 4, 12, 20]\n    assert f5([1, 2, 3]) == [2, 6]\n    assert f5([3, 2, 1]) == [2, 2]\n    assert f5([3, 2, 1, 0, 4]) == [2, 2, 0, 16]\n    assert f5([1]) == []\n    \"\"\"\n\n    return [(i * x) for i, x in enumerate(xs)][1:]\n"
instruction: str = "# Translate function f5 from Python to Rust. You are coding."
prompt_edit = prompt + instruction

In [55]:
completion = generate(prompt_edit, 128, 0.8)

In [56]:
print(completion)

from typing import List

def f1(numbers: List[float], threshold: float) -> bool:
"""
f1: List[float] -> float -> bool
    assert f1([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert f1([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert f1([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
"""
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False

// Rust f1 translation
fn has_close_elements(numbers: Vec<f64>, threshold: f64) -> bool {
    for idx in 0..numbers.len() {
        for idx2 in 0..numbers.len() {
            if idx != idx2 {
                let distance = (numbers[idx] - numbers[idx2]).abs();
                if distance < threshold {
                    return true;
                }
            }
        }
    }

    return false;
}

from typing import List

def f2(paren_string